In [25]:
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1015. Fire Sufficient Similarity/3. Analyses/2. Data Processing/Output'
cur_date = '012825'

library(missForest)
library(readxl)
library(tidyverse)
library(imputeLCMD)
library(factoextra)
# library(vegan)
# library(preprocessCore)

# reading in files
ws_df = data.frame(Data = 'WS', read_excel("Input/Woodsmoke_Data_012825.xlsx", sheet = 2)) %>%
    # removing extra replicates for samples that weren't measured at all
    filter(Value == 'NA' & Replicate == 1 | Value != 'NA') %>%
    select(-Sample_Number)
wf_df = data.frame(Data = 'WF', read_excel("Input/Wildfire_Data_012825.xlsx", sheet = 2)) %>%
    # removing extra replicates for samples that weren't measured at all
    filter(Value == 'NA' & Replicate == 1 | Value != 'NA') %>%
    select(-Sample_Number)

In [26]:
head(ws_df)
head(wf_df)

,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,267091,Forchhammer et al. 2012,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WF,1289821,Liang et al. 2021,1,PAH,Volume,NA,1-(10-methylanthracen-9-yl)ethanone,5
2,WF,1289821,Liang et al. 2021,1,PAH,Volume,DTXSID50176885,1-Acenaphthenone,1
3,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
4,WF,1289739,Campos et al. 2019,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
5,WF,1289777,Harper et al. 2019,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
6,WF,981013,Jalava et al. 2006,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA


First need to save the non-detect data.

In [27]:
nd_ws_df = ws_df %>%
    filter(Value == 'ND') %>%
    unique()

head(nd_ws_df)

,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID9020877,1-Methylnaphthalene,ND
2,WS,299223,Kasurinen et al. 2015,2,PAH,Weight,DTXSID9020877,1-Methylphenanthrene,ND
3,WS,299223,Kasurinen et al. 2015,3,PAH,Weight,DTXSID9020877,1-Methylphenanthrene,ND
4,WS,429188,Kasurinen et al. 2018,4,PAH,Weight,DTXSID0025654,1-Methylpyrene,ND
5,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID4020878,2-Methylnaphthalene,ND
6,WS,821855,Erlandsson et al. 2020,2,PAH,Weight,DTXSID4020878,2-Methylnaphthalene,ND


# Chemical/Metal Background Filter

Determining what variables have missing data for all variables delineated based on the dfs the data will be split into and analyzed. A variable will be removed if it has less than 50% of woodsmoke (WS) and wildfire (WF) experimental and/or non-detect data.

In [28]:
# # changing cols to numeric
# ws_df$Value = as.numeric(ws_df$Value)
# wf_df$Value = as.numeric(wf_df$Value)

# combining dfs
combined_df = rbind(ws_df, wf_df)
head(combined_df)

,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,267091,Forchhammer et al. 2012,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


In [35]:
variable_presence_df = combined_df %>%
    # if the value isn't MAR count it as being present
    mutate(count = ifelse(Value != 'NA', 1, 0)) %>%
    group_by(Metric, Name) %>%
    # counting number of records for each variable
    mutate(group_count = n()) %>%
    # calculating the percentage of variables with data
    reframe(Variable_Presence_Percentage = (sum(count)/group_count) * 100) %>%
    arrange(Variable_Presence_Percentage)

variable_presence_df %>%
    filter(Variable_Presence_Percentage < 50) %>%
    unique()

Metric,Name,Variable_Presence_Percentage
<chr>,<chr>,<dbl>
Volume,"1,4 Anthraquinone",0
Volume,1-Methylanthracene,0
Volume,1-Methylfluoranthene,0
Volume,1-Methylfluorene,0
Volume,1-Methylnaphthalene,0
Volume,"2,3,5-Trimethylnaphthalene",0
Volume,"2,3-Dimethylnaphthalene",0
Volume,2-Methylchrysene,0
Volume,2-Naphthaldehyde,0
